In [1]:
import os, glob
import pandas as pd
import pyranges as pr
from utils.ref import get_tss_pyranges

In [2]:
pth = os.path.join('..', '..', '..', 'data')
pth_in = os.path.join(pth, 'unzip', 'GSE230319')
pth_mod = os.path.join(pth, 'features', 'biomart', 'modules')
pth_out = os.path.join(pth, 'DELAY', 'rRNAModifications')
pth_ref = os.path.join(pth, 'ref')

In [3]:
g = pd.read_csv(os.path.join(pth_mod, 'union.csv'))
gdict = g.set_index('hsapiens').mmusculus.to_dict()
fn = os.path.join(pth_ref, 'hg38.refGene.gtf.gz')
tss = get_tss_pyranges(fn)

/home/caleb/MesenCoder/DELAY/preprocessing/ground-truth/utils/ref.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(get_upstream_tss))


In [8]:
g1, g2 = [], []
peaks_dict = {'*TWIST1_*' : 'Twist1', '*ALX1*' : 'Alx1', '*ALX4*' : 'Alx4'}
for pat, src in peaks_dict.items():
    for fn in sorted(glob.glob(os.path.join(pth_in, pat))):
        print(src, fn)
        peaks = pr.read_bed(fn)
        tss_peaks = peaks.nearest(tss, apply_strand_suffix = True)
        g_peaks = tss_peaks.df.gene_id.unique()
        tgt = [gdict[_g_] for _g_ in g_peaks if _g_ in gdict]
        g1.extend([src] * len(tgt)); g2.extend(tgt)

# write output
refNet = pd.DataFrame({'Gene1' : g1, 'Gene2' : g2}).drop_duplicates()
refNet.to_csv(os.path.join(pth_out, 'refNetwork.csv'), index = False)
refNet

Twist1 ../../../data/unzip/GSE230319/GSM7213613_TWIST1FV_dTAG-0h_r1_TWIST1_peaks.narrowPeak.gz


Twist1 ../../../data/unzip/GSE230319/GSM7213615_TWIST1FV_dTAG-0h_r2_TWIST1_peaks.narrowPeak.gz
Alx1 ../../../data/unzip/GSE230319/GSM7213651_ALX1FV_dTAG-0h_r1_V5_peaks.narrowPeak.gz
Alx1 ../../../data/unzip/GSE230319/GSM7213653_ALX1FV_dTAG-0h_r2_V5_peaks.narrowPeak.gz
Alx4 ../../../data/unzip/GSE230319/GSM7213748_TWIST1FV_dTAG-0h_CnR_ALX4_peaks.narrowPeak.gz


,Gene1,Gene2
0,Twist1,Atad3a
1,Twist1,Eva1b
2,Twist1,Vcam1
3,Twist1,Crabp2
4,Twist1,Ddr2
...,...,...
746,Alx4,Sulf2
747,Alx4,Snai1
748,Alx4,Psmg1
749,Alx4,Cited1


In [9]:
n_edges = refNet.groupby('Gene1').size()
density = (n_edges / g.shape[0])
density.sort_values(ascending = False)

Gene1
Twist1    0.672956
Alx4      0.211740
Alx1      0.092243
dtype: float64